In [ ]:
# 布林带策略
# 数据：沪深300 日频数据 from JoinQuant
# 策略核心：
# 当收盘价'Close'上穿上轨线，买入
# 当收盘价'Close'下穿下轨线，卖空

# 定义上中下轨道 mid & up & down

In [1]:
import pandas as pd
import tushare as ts
import jqdatasdk
import os
import numpy as np

In [14]:
# 读取本地下载好的沪深300_daily csv文件并进行数据预处理
df_300 = pd.read_csv("000300_daily.csv", header=0, names=["DateTime", "Open", "Close", "High", "Low", "Volume", "Money"])
df_300.head()

,DateTime,Open,Close,High,Low,Volume,Money
0,2012-09-03,2201.12,2228.37,2236.56,2197.71,3.854859e+09,3.529918e+10
1,2012-09-04,2231.72,2204.41,2234.88,2199.51,3.485269e+09,3.322020e+10
2,2012-09-05,2199.74,2199.88,2209.54,2186.96,3.475182e+09,3.044517e+10
3,2012-09-06,2204.44,2217.82,2218.68,2199.86,3.169588e+09,2.687332e+10
4,2012-09-07,2233.91,2317.18,2341.52,2233.91,9.452976e+09,8.430225e+10


In [35]:
# 算法 Mid = 最高价 + 最低价 + 收盘价 / 3 作为中轨
df_300['Tp'] =round((df_300['High'] + df_300['Low'] + df_300['Close']) / 3, 2) # Tp 当前价格 保留两位
df_300['Mid'] = np.nan # 中轨线=N日tp的移动平均线 均值拟合：N=20 20天移动平均
df_300['Up'] = np.nan  # 上轨线=中轨线+M倍的标准差(中轨线标准差) M=2 2倍标准差
df_300['Down'] = np.nan # 下轨线=中轨线-M倍的标准差（中轨线标准差）M=2 2倍标准差
df_300.head()  

,DateTime,Open,Close,High,Low,Volume,Money,Tp,Mid,Up,Down
0,2012-09-03,2201.12,2228.37,2236.56,2197.71,3.854859e+09,3.529918e+10,2220.88,NaN,NaN,NaN
1,2012-09-04,2231.72,2204.41,2234.88,2199.51,3.485269e+09,3.322020e+10,2212.93,NaN,NaN,NaN
2,2012-09-05,2199.74,2199.88,2209.54,2186.96,3.475182e+09,3.044517e+10,2198.79,NaN,NaN,NaN
3,2012-09-06,2204.44,2217.82,2218.68,2199.86,3.169588e+09,2.687332e+10,2212.12,NaN,NaN,NaN
4,2012-09-07,2233.91,2317.18,2341.52,2233.91,9.452976e+09,8.430225e+10,2297.54,NaN,NaN,NaN


In [40]:
# 求中轨线Tp方法 1 for循环
# for i in range(19, len(df_300)):
#     df_300.loc[df_300.index[i], 'Mid'] = df_300['Tp'][i-19:i+1].mean()

# 求中轨线Tp方法 2 rolling(n) 窗口滑动求平均
# df_300['Mid'] = round(df_300['Tp'].rolling(20).mean(), 2)
# df_300['Up'] = round((df_300['Tp'].rolling(20).std()) * 2 + df_300['Tp'].rolling(20).mean(), 2)
# df_300['Down'] = round(df_300['Tp'].rolling(20).mean() - (df_300['Tp'].rolling(20).std()) * 2, 2)

# 写成函数形式
'''
该函数用于计算布林带的上中下轨道
Param: DataFrame; N; m
'''
def construct_Band(df, N, m): # Dataframe, N = days of moving average, M = std倍数
    df['Mid'] = round(df['Tp'].rolling(N).mean(), 2)
    df['Up'] = round((df['Tp'].rolling(N).std()) * 2 + df['Tp'].rolling(N).mean(), 2)
    df['Down'] = round(df['Tp'].rolling(N).mean() - (df['Tp'].rolling(N).std()) * 2, 2)
    return df


In [42]:
'''20日移动均线和两倍标准差构建BBand'''
construct_Band(df_300, 20, 2)
df_300.head(40)

,DateTime,Open,Close,High,Low,Volume,Money,Tp,Mid,Up,Down
0,2012-09-03,2201.12,2228.37,2236.56,2197.71,3.854859e+09,3.529918e+10,2220.88,NaN,NaN,NaN
1,2012-09-04,2231.72,2204.41,2234.88,2199.51,3.485269e+09,3.322020e+10,2212.93,NaN,NaN,NaN
2,2012-09-05,2199.74,2199.88,2209.54,2186.96,3.475182e+09,3.044517e+10,2198.79,NaN,NaN,NaN
3,2012-09-06,2204.44,2217.82,2218.68,2199.86,3.169588e+09,2.687332e+10,2212.12,NaN,NaN,NaN
4,2012-09-07,2233.91,2317.18,2341.52,2233.91,9.452976e+09,8.430225e+10,2297.54,NaN,NaN,NaN
5,2012-09-10,2320.56,2326.67,2334.50,2313.57,6.533055e+09,6.325204e+10,2324.91,NaN,NaN,NaN
6,2012-09-11,2316.53,2311.89,2316.53,2294.82,4.626072e+09,4.338977e+10,2307.75,NaN,NaN,NaN
7,2012-09-12,2325.93,2320.07,2337.20,2298.30,4.962252e+09,4.648517e+10,2318.52,NaN,NaN,NaN
8,2012-09-13,2316.68,2298.46,2325.53,2298.37,4.204985e+09,3.802400e+10,2307.45,NaN,NaN,NaN
9,2012-09-14,2324.01,2315.54,2332.42,2298.83,6.157181e+09,5.543861e+10,2315.60,NaN,NaN,NaN
